<a href="https://colab.research.google.com/github/almc6742/cu-git-workshop/blob/master/Exercise_1_Basic_Ad_Analyses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
fbpony = 'drive/My Drive/Colab Notebooks/Travel Pony Facebook.csv'
df = pd.read_csv(fbpony)

First, create the analytics 'cost per impression' by dividing Amount Spent / Impressions. Do this for each ad in your data, and save that as a column.

In [0]:
df['Cost per Impression'] = df['Amount Spent (USD)']/df['Impressions']

When considering 'cost per impression', what day of the week works best? (What day is it cheapest to generate impressions)

What day works worst? (What day is most expensive)

In [0]:
df['Start Date'] = pd.to_datetime(df['Start Date'])

df['Day of Week'] = df['Start Date'].dt.day_name()

In [0]:
df.groupby('Day of Week', as_index=False)['Cost per Impression'].mean()


,Day of Week,Cost per Impression
0,Friday,0.004097
1,Monday,0.002910
2,Saturday,0.002629
3,Sunday,0.003688
4,Thursday,0.003398
5,Tuesday,0.003040
6,Wednesday,0.002981


* Best Day (cheapest): Saturday
* Worst Day (most expensive): Friday

---
Next, compute the correlation between Amount Spent and the following variables: Reach, Frequency, Unique Clicks, Page Likes.

Which correlation is the strongest? What does that mean practically?


In [0]:
df1 = df[['Amount Spent (USD)', 'Reach', 'Frequency', 'Unique Clicks', 'Page Likes']].copy()

In [0]:
df1.corr()

,Amount Spent (USD),Reach,Frequency,Unique Clicks,Page Likes
Amount Spent (USD),1.000000,0.703124,0.130201,0.882993,0.757612
Reach,0.703124,1.000000,0.334101,0.722249,0.304388
Frequency,0.130201,0.334101,1.000000,0.135103,0.000182
Unique Clicks,0.882993,0.722249,0.135103,1.000000,0.584614
Page Likes,0.757612,0.304388,0.000182,0.584614,1.000000


The strongest correlation is between Amount Spent and Unique Clicks. This means that as Travel Pony increases spend on their ads, the number of unique clicks is expected to increase significantly and vice versa.

---
Finally, perform a simple multiple regression analysis where Unique Clicks is the dependent variable and Reach and Frequency are the independent (predictor) variables.

What variable most strongly predicts unique clicks? What does that mean practically?

In [0]:
pivoteddata = df1.pivot_table(index='Amount Spent (USD)', aggfunc='mean')

target = pivoteddata['Reach']
dep = pivoteddata['Unique Clicks']

mod = sm.OLS(target, dep)
res = mod.fit()
print(res.summary())

                                 OLS Regression Results                                
Dep. Variable:                  Reach   R-squared (uncentered):                   0.769
Model:                            OLS   Adj. R-squared (uncentered):              0.769
Method:                 Least Squares   F-statistic:                              3769.
Date:                Thu, 14 Nov 2019   Prob (F-statistic):                        0.00
Time:                        01:05:04   Log-Likelihood:                         -10965.
No. Observations:                1131   AIC:                                  2.193e+04
Df Residuals:                    1130   BIC:                                  2.194e+04
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                    coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------

In [0]:
target = pivoteddata['Frequency']
dep = pivoteddata['Unique Clicks']

mod = sm.OLS(target, dep)
res = mod.fit()
print(res.summary())

                                 OLS Regression Results                                
Dep. Variable:              Frequency   R-squared (uncentered):                   0.308
Model:                            OLS   Adj. R-squared (uncentered):              0.307
Method:                 Least Squares   F-statistic:                              503.0
Date:                Thu, 14 Nov 2019   Prob (F-statistic):                    1.90e-92
Time:                        01:05:04   Log-Likelihood:                         -1421.8
No. Observations:                1131   AIC:                                      2846.
Df Residuals:                    1130   BIC:                                      2851.
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                    coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------

Reach most strongly predicts Unique Clicks. This means that as the number of people who see your ad content increases, the unique clicks is expected to have a similar lift.